In [2]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Create Validation set
x_train, x_val = x_train[:45000], x_train[45000:]
y_train, y_val = y_train[:45000], y_train[45000:]

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_val = (x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_val = np_utils.to_categorical(y_val,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [3]:
# Activation functions
def swish(x):
    return x*K.sigmoid(x)

def e_swish_1(x):
    return K.maximum(0, 2*x-swish(x))

def e_swish_2(x):
    return K.maximum(swish(x), 2*x-swish(x))

def e_swish_3(x):
    return K.maximum((1+0.25/(K.exp(K.abs(x))-0.5))*swish(x), 2*x-swish(x))

In [4]:
def create(act):
    baseMapNum = 32
    weight_decay = 1e-4

    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(4*2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.33))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [5]:
activations = [e_swish_2, "relu",  swish,  e_swish_3, e_swish_1]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_2', 'relu', 'swish', 'e_swish_3', 'e_swish_1']


In [6]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [10]:
# Create histories
histories = {}
for act_name in names:
    histories[act_name] = []

In [7]:
#training
batch_size = 128
epochs=1

for i, act in enumerate(activations[:1]):
    model = create(act)
    opt = keras.optimizers.SGD(lr=0.000001)
    model.load_weights("cifar10_normal_rms_ep175e_swish_2.h5")
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                              steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,
                              verbose=1,validation_data=(x_val,y_val))
    print(his.history)
    model.save_weights('cifar10_avg_5_fuck_it.h5')
    
    scores = model.evaluate(x_test, y_test)
    print(scores)

Epoch 1/1
351/351 [==============================] - 253s 720ms/step - loss: 0.2878 - acc: 0.9475 - val_loss: 0.1568 - val_acc: 0.9934
{'val_acc': [0.99339999999999995], 'loss': [0.28774476522089304], 'val_loss': [0.15684286861419677], 'acc': [0.94747281159536789]}
10000/10000 [==============================] - 19s 2ms/step
[0.41527074699401856, 0.91600000000000004]


In [9]:
model = create(act)
opt = keras.optimizers.SGD(lr=0.000001)
model.load_weights("cifar10_avg_5e_swish_2_0.h5")
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

scores = model.evaluate(x_test, y_test)
print(scores)

10000/10000 [==============================] - 19s 2ms/step
[0.4151816153526306, 0.91569999999999996]


In [ ]:
# Get average score 
scores = []
for act_name in names:
    total = 0
    his = histories[act_name]
    for record in his:
        total = total + record["val_acc"][-1]
    scores.append(total/5)